<a href="https://colab.research.google.com/github/camilasamurai/KOproject/blob/main/KO_URI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdflib
!pip install pandas

     |████████████████████████████████| 482 kB 26.1 MB/s 
     |████████████████████████████████| 41 kB 748 kB/s 


In [2]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace # basic RDF handling
from rdflib.namespace import FOAF , XSD, RDF, RDFS # most common namespaces
import pandas as pd
import re
import unicodedata

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/camilasamurai/KOproject/main/Article.csv', sep=',', encoding='ISO-8859-1')  

In [20]:
df

,Subject,Predicate,Object
0,Article,dcterm: title,Frida Kahlo y el arte mexicano
1,Article,dcterm: type,Journal Article
2,Article,dcterm: created,1943
3,Article,dcterm: language,Spanish
4,Article,CRM: P55 has current location,Instituto de investigaciones Bibliográficas
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [21]:
word_regex_pattern = re.compile("[^A-Za-z0-9]+")

def camel(chars): # URI in Camel Case - useful for english version
  words = word_regex_pattern.split(chars)
  return "".join(w.lower().strip() if i is 0 else w.title() for i, w in enumerate(words))

def diacritics(chars): # get rid of diacritics and normalise text - useful for italian language
  normalized = unicodedata.normalize("NFKD", chars)
  return "".join(c for c in normalized if unicodedata.category(c) != "Mn")

In [22]:
g = Graph()

In [25]:
efrbroo = Namespace("http://erlangen-crm.org/efrbroo/") 
frida = Namespace("https://w3id.org/fridalod/")
dct = Namespace("http://purl.org/dc/terms/")
ecrm = Namespace("http://erlangen-crm.org/current/")

In [26]:
g.bind('efrbroo', efrbroo)
g.bind('fridalod', frida)
g.bind('dct', dct)
g.bind('ecrm', ecrm)

In [29]:
for (idx, row) in df.iterrows():
  # first, we work on titles and ids in df
  subject = (str(row.Subject).strip())
  Object =(str(row.Object).strip())

  # we create first the URIRefs i will put in the triples 
  item = URIRef(frida + 'item' + subject)
  person = URIRef(frida + 'person' + subject)
  date = URIRef(frida + 'date' + subject)

  # now we can create the triples reusing the schema we choose and the URIs we created
  # first we declare that an entity belongs to a class
  g.add((item, RDF.type, efrbroo.F4_Manifestation_Singleton))
  g.add((person, RDF.type, efrbroo.F2_Expression))
  g.add((date, RDF.type, efrbroo.F28_Expression_Creation))

  # we go on with adding triples to the graph but creating also relations between entities
  g.add((item, efrbroo.R17_created, item))
  g.add((person, efrbroo.R18_created, person))
  g.add((date, dct.title, (Literal(date, datatype=XSD.string))))
  
for s,p,o in g.triples((None, None, None)):
  print(s,p,o)

https://w3id.org/fridalod/itemArticle http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://erlangen-crm.org/efrbroo/F4_Manifestation_Singleton
https://w3id.org/fridalod/personArticle http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://erlangen-crm.org/efrbroo/F2_Expression
https://w3id.org/fridalod/itemnan http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://erlangen-crm.org/efrbroo/F4_Manifestation_Singleton
https://w3id.org/fridalod/dateArticle http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://erlangen-crm.org/efrbroo/F28_Expression_Creation
https://w3id.org/fridalod/datenan http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://erlangen-crm.org/efrbroo/F28_Expression_Creation
https://w3id.org/fridalod/personArticle http://erlangen-crm.org/efrbroo/R18_created https://w3id.org/fridalod/personArticle
https://w3id.org/fridalod/dateArticle http://purl.org/dc/terms/title https://w3id.org/fridalod/dateArticle
https://w3id.org/fridalod/personnan http://erlangen-crm.org/efrbroo/R

In [36]:
# In this case, for each category, we want to create a label and a URI

for (idx, row) in df.iterrows():

  # We redifine expression, so we can match each expression to the respective(s) cateogory(ies)
  subject = URIRef(subject+ 'item' + str(row.Subject).strip())

  cat_list = []
  cat_list = (str(row.Object).split(','))

  for c in cat_list:
    uri_partial = diacritics(c) # the function we defined previousely
    uri = camel(uri_partial)
    label = c.strip()

    # again, we create entities
    categ_uri = URIRef(frida + uri)

    # we add the labels
    g.add((categ_uri, RDFS.label, (Literal(label, datatype=XSD.string))))

    # define them and make connections between them 
    g.add((categ_uri, RDF.type, ecrm.E1_CRM_Entity)) # we define the category 
    g.add((subject, ecrm.P67_refers_to, categ_uri)) # now we say that the item (expression) represents a category 

for s,p,o in g.triples((None, ecrm.P67_refers_to, None)):   
  print(s,p,o)

for s,p,o in g.triples((None, RDFS.label, None)):   
  print(s,p,o)


nanitemArticleitemArticleitemArticle http://erlangen-crm.org/current/P67_refers_to https://w3id.org/fridalod/fridaKahloYElArteMexicano
nanitemArticleitemArticleitemArticleitemArticleitemArticleitemArticleitemArticleitemnanitemArticleitemArticleitemArticleitemArticleitemArticleitemnanitemArticle http://erlangen-crm.org/current/P67_refers_to https://w3id.org/fridalod/fridaKahloYElArteMexicano
nanitemArticleitemArticleitemArticleitemArticle http://erlangen-crm.org/current/P67_refers_to https://w3id.org/fridalod/journalArticle
nanitemArticleitemArticleitemArticleitemArticleitemArticleitemArticleitemArticleitemnanitemArticleitemArticleitemArticleitemArticleitemArticleitemnanitemArticleitemArticle http://erlangen-crm.org/current/P67_refers_to https://w3id.org/fridalod/journalArticle
nanitemArticleitemArticleitemArticleitemArticleitemArticle http://erlangen-crm.org/current/P67_refers_to https://w3id.org/fridalod/1943
nanitemArticleitemArticleitemArticleitemArticleitemArticleitemArticleitemArt

In [38]:
# we save the RDF in Turtle serialisation
output_path = "/content/sample_data/fridaTurtle.ttl"

file = g.serialize(output_path, format='ttl')
# print(g.serialize(format='ttl'))

